In [1]:
#Install gurobi dependency by uncommenting following line:
!pip install gurobipy numpy pandas

You should consider upgrading via the 'C:\Users\artur\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# A Transportation Problem

## Problem Description

A boutique brewery has two warehouses from which it distributes beer to five carefully chosen bars. At the start of every week, each bar sends an order to the brewery’s head office for so many crates of beer, which is then dispatched from the appropriate warehouse to the bar. The brewery would like to have an interactive computer program which they can run week by week to tell them which warehouse should supply which bar so as to minimize the costs of the whole operation. For example, suppose that at the start of a given week the brewery has 1000 cases at warehouse A, and 4000 cases at warehouse B, and that the bars require 500, 900, 1800, 200, and 700 cases respectively. Which warehouse should supply which bar?

## Formulation
For transportation problems, using a graphical representation of the problem is often helpful during formulation. Here is a graphical representation of The Beer Distribution Problem.

![brewery_nodes.jpg](https://www.coin-or.org/PuLP/_images/brewery_nodes.jpg)

## Identify the Decision Variables

In transportation problems we are deciding how to transport goods from their supply nodes to their demand nodes. The decision variables are the Arcs connecting these nodes, as shown in the diagram below. We are deciding how many crates of beer to transport from each warehouse to each pub.

![brewery_arcs.jpg](https://www.coin-or.org/PuLP/_images/brewery_arcs.jpg)

Let:
- A1 = number of crates of beer to ship from Warehouse A to Bar 1
- A5 = number of crates of beer to ship from Warehouse A to Bar 5
- B1 = number of crates of beer to ship from Warehouse B to Bar 1
- B5 = number of crates of beer to ship from Warehouse B to Bar 5

![6e22bc037817c4d46d068fe3d24df3246fbdf4f8.png](https://www.coin-or.org/PuLP/_images/math/6e22bc037817c4d46d068fe3d24df3246fbdf4f8.png)


The lower bound on the variables is Zero, and the values must all be Integers (since the number of crates cannot be negative or fractional). There is no upper bound.

## Formulate the Objective Function

The objective function has been loosely defined as cost. The problem can only be formulated as a linear program if the cost of transportation from warehouse to pub is a linear function of the amounts of crates transported. Note that this is sometimes not the case. This may be due to factors such as economies of scale or fixed costs. For example, transporting 10 crates may not cost 10 times as much as transporting one crate, since it may be the case that one truck can accommodate 10 crates as easily as one. Usually in this situation there are fixed costs in operating a truck which implies that the costs go up in jumps (when an extra truck is required). In these situations, it is possible to model such a cost by using zero-one integer variables.

We shall assume then that there is a fixed transportation cost per crate. (If the capacity of a truck is small compared with the number of crates that must be delivered then this is a valid assumption). Lets assume we talk with the financial manager, and she gives us the following transportation costs (dollars per crate):

|From Warehouse to Bar|A|B|
| --- | --- | --- |
|1|2|3|
|2|4|1|
|3|5|3|
|4|2|2|
|5|1|3|

Minimise the Transporting Costs = Cost per crate for RouteA1 * A1 (number of crates on RouteA1) + ... + Cost per crate for RouteB5 * B5 (number of crates on RouteB5)

![6fec37a5f986724e3414e35e56081ae1e1745115.png](https://www.coin-or.org/PuLP/_images/math/6fec37a5f986724e3414e35e56081ae1e1745115.png)

## Formulate the Constraints

The constraints come from considerations of supply and demand. The supply of beer at warehouse A is 1000 cases. The total amount of beer shipped from warehouse A cannot exceed this amount. Similarly, the amount of beer shipped from warehouse B cannot exceed the supply of beer at warehouse B. The sum of the values on all the arcs leading out of a warehouse, must be less than or equal to the supply value at that warehouse:

Such that:

- A1 + A2 + A3 + A4 + A5 <= 1000
- B1 + B2 + B3 + B4 + B5 <= 4000

![0e57edac5e6f1152e622250b16c332b084d62c40.png](https://www.coin-or.org/PuLP/_images/math/0e57edac5e6f1152e622250b16c332b084d62c40.png)

The demand for beer at bar 1 is 500 cases, so the amount of beer delivered there must be at least 500 to avoid lost sales. Similarly, considering the amounts delivered to the other bars must be at least equal to the demand at those bars. Note, we are assuming there are no penalties for oversupplying bars (other than the extra transportation cost we incur). We can _balance_ the transportation problem to make sure that demand is met exactly - there will be more on this later. For now, the sum of the values on all the arcs leading into a bar, must be greater than or equal to the demand value at that bar:


- A1 + B1 >= 500
- A2 + B2 >= 900
- A3 + B3 >= 1800
- A4 + B4 >= 200
- A5 + B5 >= 700

![cc15edd9db6d9dbab572c9441f343b131262afd2.png](https://www.coin-or.org/PuLP/_images/math/cc15edd9db6d9dbab572c9441f343b131262afd2.png)

Finally, we have already specified the amount of beer shipped must be non-negative.

## Gurobi Model

First, start your Python file with a heading and the import Gurobi statement:

In [2]:
"""
The Beer Distribution Problem for the Gurobi
"""

# import gurobi library
import gurobipy as gp         #Gurobi Python interface
from gurobipy import GRB      #Import as shortcut to avoid writing GP.grb
from itertools import product #product creates the cartesian product from 2 or more lists.

The start of the formulation is a simple definition of the nodes and their limits/capacities. The node names are put into lists, and their associated capacities are put into dictionaries with the node names as the reference keys:

In [3]:
# Creates a list of all the supply nodes
Warehouses = ["A","B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
          "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 500,
          "2": 900,
          "3": 1800,
          "4": 200,
          "5": 700}

Then we create a dictonary with as key as tuple with (Bar, Warehouse) and as value the the shipping cost.
if costs[(“A”,“1”)] is called it will return the cost of from warehouse A to bar 1.

In [4]:
costs = {('A', '1'): 2,
         ('A', '2'): 4,
         ('A', '3'): 5,
         ('A', '4'): 2,
         ('A', '5'): 1,
         ('B', '1'): 3,
         ('B', '2'): 1,
         ('B', '3'): 3,
         ('B', '4'): 2,
         ('B', '5'): 3}

The prob variable is created using the LpProblem function, with the usual input parameters.

In [5]:
# Creates the prob variable to contain the problem data
m =gp.Model('BeerDistributionProblem')

Restricted license - for non-production use only - expires 2024-10-28


A list of tuples is created containing all the arcs.

In [6]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]

In [7]:
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5')]

A dictionary called route_var is created which contains the LP variables. The reference keys to the dictionary are the warehouse name, then the bar name as as tuple. By setting the data-type with vtype as an Integer we restrict the decision variables to be 0 or higher.

In [8]:
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = m.addVars(Routes, vtype=GRB.INTEGER, name='Route')
route_vars

{('A', '1'): <gurobi.Var *Awaiting Model Update*>,
 ('A', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('A', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('A', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('A', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('B', '1'): <gurobi.Var *Awaiting Model Update*>,
 ('B', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('B', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('B', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('B', '5'): <gurobi.Var *Awaiting Model Update*>}

The objective function is added to the variable prob using a list comprehension. Since route_vars and costs are now dictionaries (with further internal dictionaries), they can be used as if they were tables, as for (w,b) in Routes will cycle through all the combinations/arcs. Note that i and j could have been used, but w and b are more meaningful.

In [9]:
#Set the objective function as the product of the number of units transported on a route with the cost of that route.
m.setObjective(route_vars.prod(costs),GRB.MINIMIZE)

The supply and demand constraints are added using a normal for loop and a list comprehension. Supply Constraints: For each warehouse in turn, the values of the decision variables (number of beer cases on arc) to each of the bars is summed, and then constrained to being less than or equal to the supply max for that warehouse. Demand Constraints: For each bar in turn, the values of the decision variables (number on arc) from each of the warehouses is summed, and then constrained to being greater than or equal to the demand minimum.

In [10]:
for w in Warehouses:
    # route_vars.sum(w) sums up all decision variables with a tuple that starts with the value of w
    # Eg route_vars.sum('A') is the sum of all transported units from warehouse A
    m.addConstr(route_vars.sum(w)<=supply[w], name=f"SupplyConstraintWarehouse{w}")
for b in Bars:
    # route_vars.sum('*',b) sums up all decision variables with a tuple
    #    where the second value is equal to the value of b
    # Eg route_vars.sum('*','1') is the sum of all transported units to bar 1
    m.addConstr(route_vars.sum('*',b)>=demand[b], name=f"DemandBar{b}")

Solve the model

In [11]:
# Run the model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x183ef47f
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 4e+03]
Presolve removed 7 rows and 10 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 8600 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.600000000000e+03, best bound 8.600000000000e+03, gap 0.0000%


In [12]:
print(f"Optimal objective value: {m.objVal}")

print("\nShipment plan:")
for warehouse, bar in route_vars.keys():
    if (abs(route_vars[warehouse, bar].x) > 0): #Only print if not 0
        print (f"Ship {route_vars[warehouse, bar].x} units to bar {bar} from warehouse {warehouse}")

Optimal objective value: 8600.0

Shipment plan:
Ship 300.0 units to bar 1 from warehouse A
Ship 700.0 units to bar 5 from warehouse A
Ship 200.0 units to bar 1 from warehouse B
Ship 900.0 units to bar 2 from warehouse B
Ship 1800.0 units to bar 3 from warehouse B
Ship 200.0 units to bar 4 from warehouse B


*CC-BY-SA pulp documentation team*

https://www.coin-or.org/PuLP/CaseStudies/a_transportation_problem.html

https://creativecommons.org/licenses/by-sa/3.0/nz/